In [1]:
# Notes for configuring the presentation:
# reveal/rise config is in the notebook metadata (see Edit - Edit Notebook Metadata, "rise" section)
# possible themes: https://revealjs.com/themes/
# - for us: simple, white
# possible transitions: https://revealjs.com/transitions/
# - for us: none, slide, zoom
# More: see https://github.com/damianavila/RISE/blob/master/doc/customize.md#how-to-customize
# Documentation: https://rise.readthedocs.io/en/latest/

## <center>An Overview of<br>Python GateNLP<br></center>

* Online slides: https://gatenlp.github.io/python-gatenlp/overview/gatenlp-overview.slides.html
* Slides License: [CC BY-NC-SA 3.0](https://creativecommons.org/licenses/by-nc-sa/3.0/)

# Python GateNLP: Aims

* NLP framework written in pure Python. 
* Similar concepts as Java GATE: documents, document features, annotation sets, annotations, ...
* "pythonic" API, try to make basic things very simple (e.g. loading/saving of documents)
* if possible, fix some of the problems/idiosyncracies of Java GATE
* Does NOT try to be a full multilingual NLP processing package, rather COMBINE:
  * Use existing tools and solutions: Spacy, Stanford Stanza, NLTK, ...
  * Add own tools and improvements where needed
  * Concentrate on what others do NOT provide


# Python GateNLP: status

* Current release: 1.0.6
* Published as PyPi package
  * 400 - 1400 downloads / month, 40.000 total (see [PePy](https://pepy.tech/project/gatenlp))
  * 40.000 downloads total
* All 1.0.x: get community feedback:
  * how to improve API, abstractions, conventions, find bugs
  * what is most important to still get added?
* Planned 1.1.x releases and onwards: stable API

# Python GateNLP: Info and Feedback

* Documentation: https://gatenlp.github.io/python-gatenlp/
* Sources: https://github.com/GateNLP/python-gatenlp
* Report a bug, request a feature with issue tracker: https://github.com/GateNLP/python-gatenlp/issues
* Discuss, ask: 
  * discussions forum at https://github.com/GateNLP/python-gatenlp/discussions
  * GATE mailing list https://groups.io/g/gate-users
* [ Developers Chat: https://gitter.im/GateNLP/python-gatenlp ]

# Python GateNLP: Main Concepts

* [**Document**](https://gatenlp.github.io/python-gatenlp/documents): has text,  named [**AnnotationSets**](https://gatenlp.github.io/python-gatenlp/annotationsets), each with  [**Annotations**](https://gatenlp.github.io/python-gatenlp/annotations),  Annotations and AnnotationSets can have **Features**
* **Annotators** process a document, return a document or a (possibly empty) list of documents: [**Gazetteers**](https://gatenlp.github.io/python-gatenlp/gazetteers) (StringGazetteer, [**StringRegexAnnotator**](https://gatenlp.github.io/python-gatenlp/stringregex), TokenGazetteer), [**Tokenizers**](https://gatenlp.github.io/python-gatenlp/tokenizers), [**PAMPAC**](https://gatenlp.github.io/python-gatenlp/pampac) (annotation pattern matching rules), [**Spacy annotator**](https://gatenlp.github.io/python-gatenlp/lib_spacy), [**Stanza Annotator**](https://gatenlp.github.io/python-gatenlp/lib_stanza), [**GateWorkerAnnotator**](https://gatenlp.github.io/python-gatenlp/gateworker), [**Client Annotators**](https://gatenlp.github.io/python-gatenlp/client_annotators), ...
* Collections of documents are represented as:  [**Corpus**](https://gatenlp.github.io/python-gatenlp/corpora), [**DocumentSource**](https://gatenlp.github.io/python-gatenlp/corpora) or [**DocumentDestination**](https://gatenlp.github.io/python-gatenlp/corpora)
* Pipelines, Executors: process documents, pipe through several annotators, process results
* Import/Output/Visualization: support various formats, interact via HTML

# Documents

In [5]:
from gatenlp import Document

doc1 = Document("This is a small test document")

doc1.features["Feature1"] = "Some feature value"
set1 = doc1.annset("Set1")

set1.add(0, 4, "Ann", dict(feat1=2, feat2="some string"))

doc1

Document(This is a small test document,features=Features({'Feature1': 'Some feature value'}),anns={'Set1': AnnotationSet([Annotation(0,4,Ann,features=Features({'feat1': 2, 'feat2': 'some string'}),id=0)])})

# Documents

In [7]:
# from IPython.display import display
doc2 = Document.load("https://en.m.wikipedia.org/wiki/Gate_(disambiguation)", fmt="html")

doc2

Document(
Gate (disambiguation) - Wikipedia
Open main menu
Home
Random
Nearby
Log in
Settings
Donate
About Wikipedia
Disclaimers
Search
Gate (disambiguation)
Article
Talk
Language
Watch
Edit
Look up GATE, Gate, gate, or -gate in Wiktionary, the free dictionary.
A gate is an opening in a wall or fence fitted with a moveable barrier allowing it to be closed.
Gate or GATE may also refer to:
Contents
1 Arts, entertainment, and media
2 Engineering
2.1 Electronics
2.2 Equipment
2.3 Hydraulics
3 Enterprises and organizations
4 Places
4.1 Famous gates
5 Scandals and conspiracy theories
6 Science and technology
7 Other uses
8 See also
Arts, entertainment, and mediaEdit
Gate (album), a 1995 album by Peter Frohmader
The Gate, a 1987 horror film
Gate (film), a 2018 South Korean film
Gate (novel series), a 2006/2010 novel series by Takumi Yanai, with comic (manga, 2011) and television (anime, 2015) adaptations
Gate (solitaire), a card game
GATE (video game), a 1991 action-adventure video game
Gåte, a Norwegian band
Gåte (2002 EP), by the eponymous band
Grammy, Academy, Tony, and Emmy Awards, or "GATE Awards", see List of people who have won Academy, Emmy, Grammy, and Tony Awards
EngineeringEdit
ElectronicsEdit
Gate (transistor), terminal of a field effect transistor
Logic gate, a functional building block in digital logic such as "and", "or", or "not"
Metal gate, the gate material in a MOSFET transistor
Noise gate, a high-quality audio squelch control for reducing noise
Range gate, the area encompassed by one pixel of radar data
EquipmentEdit
Gate, a rowing term for the hinged bar that prevents the oar from coming out
Gate, a sprue molding channel which carries molten metal into a mold
Gate, a sprung safety latch on a lifting hook or carabiner
HydraulicsEdit
Gate (hydraulic engineering), a movable structure used to control the flow of fluid in a pipe or channel
Gate (water transport), the watertight door that seals off a chamber of a lock
Floodgate, an adjustable gate used to control water flow in flood barriers, lake, river, stream, or levee systems
Gate valve, a valve that opens by lifting a wedge out of the path of the fluid
Enterprises and organizationsEdit
GATE (organization), an organization working on gender identity and sex characteristics issues
Gate Petroleum, a U.S. gas station franchise founded in Jacksonville, Florida
Gate Studios, a British film studio in operation from 1928 to the early 1950s
Gate Theatre, in Dublin
Gay Alliance Toward Equality, or "GATE", one of the first Canadian gay liberation groups
Grupo de Ações Táticas Especiais (Portuguese for "Special Tactical Actions Group"), a Brazilian police special forces squad
PlacesEdit
Gate, Arkansas
Gate, Oklahoma
Gate, Washington
Gate City, Virginia
Gate District, St. Louis
Famous gatesEdit
Gates in Jerusalem's Old City Walls
Gates of Delhi, India
Brandenburg Gate, 18th-century neoclassical monument in Berlin
Cilician Gates or Gülek Pass, a pass through the Taurus Mountains connecting the low plains of Cilicia to the Anatolian Plateau, Turkey
Gate of Kuwait, a supertall skyscraper under construction in Kuwait City, Kuwait
Golden Gate, a strait on the west coast of North America that connects San Francisco Bay to the Pacific Ocean
India Gate, the All India War Memorial, a memorial located on the Rajpath, the eastern edge of the "ceremonial axis" of New Delhi, India
Ishtar Gate, the eighth gate to the inner city of Babylon
Lion Gate, the main entrance of the Bronze Age citadel of Mycenae, southern Greece
Menin Gate, a war memorial dedicated to the British and Commonwealth soldiers who were killed in the Ypres, Belgium during World War I
Victoria Gate (Valletta), a city gate in Valletta, Malta
Scandals and conspiracy theoriesEdit
"-gate", common suffix for a public scandal (see List of scandals with "-gate" suffix) and for conspiracy theories, such as:
Gategate, a scandal in the United Kingdom involving the Government Chief Whip, the police, and a gate
Pizzagate conspiracy th

# AnnotationSets, Annotations

In [14]:
# set1 is the set that is part of the document
set1 = doc2.annset("Original markups")
print("Set has", len(set1), "annotations, immutable=", set1.immutable, "detached=", set1.isdetached())

Set has 468 annotations, immutable= False detached= False


In [18]:
# subset1 is an immutable, "detached" set. 
subset1 = set1.with_type("h1", "head", "form") 
print("Set has", len(subset1), "annotations, immutable=", subset1.immutable, "detached=", subset1.isdetached())

# Annotations have a default order: starting offset, annotation id (= order if addition)
# -> zero length annotations have a well-defined order
print("Annotations:", list(subset1))

Set has 3 annotations, immutable= True detached= True
Annotations: [Annotation(1,35,head,features=Features({}),id=2), Annotation(35,127,form,features=Features({'class': 'minerva-header', 'action': '/w/index.php', 'method': 'get'}),id=31), Annotation(127,149,h1,features=Features({'id': 'section_0'}),id=87)]


# AnnotationSet operations

In [30]:
# remove all annotations, add 2 new ones
annset=doc1.annset("Set1")
annset.clear()
annset.add(0,4,"AnnType1")
annset.add(5,7,"AnnType1", dict(a=1, b=2))
annset.add(0,20,"AnnType2")
annset.add(22,22,"AnnType3")
doc1

Document(This is a small test document,features=Features({'Feature1': 'Some feature value'}),anns={'Set1': AnnotationSet([Annotation(0,4,AnnType1,features=Features({}),id=23), Annotation(0,20,AnnType2,features=Features({}),id=25), Annotation(5,7,AnnType1,features=Features({'a': 1, 'b': 2}),id=24), Annotation(22,22,AnnType3,features=Features({}),id=26)])})

## Annotation Relations

* Annotations can overlap arbitrarily
* Annotation API has methods to check **how** they relate to each other
  * overlap, within, covering, before, after, rightoverlapping, startingat, endingwith, coextensive ...
* Annotation API implements ordering by start offset and annotation id

![Annotation Relations](data/ann-relations-cropped.png)

* Ann1 overlaps with all others, covers all but Ann2 and Ann4 
* Ann5 is directly before Ann3, is before Ann6
* Ann10 starts at Ann1, Ann12 ends with Ann1, Ann3 and Ann9 are coextensive 

## Annotation Relations

![Annotation Relations](ann-relations-cropped.png)

Let's load and view an example document to demonstrate this:

In [33]:
# Make sure the html ann viewer is smaller
from IPython.core.display import display, HTML
display(HTML("<style>#view1-wrapper { font-size: 75% !important; } #view1-row1 {height: 10em; min-height: 5em;}</style>"))


doc3 = Document.load("ann-relations.bdocjs")
doc3.show(htmlid="view1")

## Annotation/Span Relations API 

![Annotation Relations](data/ann-relations-cropped.png)

In [37]:
from gatenlp import Span

In [39]:
# make a variable for each annotation type
for anntype in list(doc3.annset("set1").type_names):
    vars()[anntype.lower()] = doc3.annset("set1").with_type(anntype).for_idx(0)
print("Ann2 isoverlapping Ann1:", ann2.isoverlapping(ann1))
print("Ann2 isbefore Ann3:", ann2.isbefore(ann3))
print("Ann3 isafter Ann2:", ann3.isafter(ann2))
print("Ann1 iscovering Ann5:", ann1.iscovering(ann5))
print("Ann3 iscoextensive Ann9:", ann3.iscoextensive(ann9))
print("Ann6 iswithin Ann1:", ann6.iswithin(ann1))
print("Ann4 isrightoverlapping Ann1:", ann4.isrightoverlapping(ann1))
print("Span(0,3).isoverlapping(ann2)):", Span(0,3).isoverlapping(Span(2,5)))

Ann2 isoverlapping Ann1: True
Ann2 isbefore Ann3: True
Ann3 isafter Ann2: True
Ann1 iscovering Ann5: True
Ann3 iscoextensive Ann9: True
Ann6 iswithin Ann1: True
Ann4 isrightoverlapping Ann1: True
Span(0,3).isoverlapping(ann2)): True


## AnnotationSet: retrieve by relation

* get all annotations that overlap/are before/start at/... an annotation/span/annotation set
* returns a new annotation set
* returned set is **detached**: not part of document, changes do set not affect document 
* returned set is initially **immutable**: set cannot be changed, can be made mutable
* annotations **are mutable** and still the same as in the set!
* possible to "detach" annotations by (deep)copying them

In [25]:
set1 = doc3.annset("set1") # "attached" set
print("Within Ann1: ", [a.type for a in set1.within(ann1)])
print("Coextensive with Ann3:", [a.type for a in set1.coextensive(ann3)])
print("Coextensive with span of Ann3:", [a.type for a in set1.coextensive(ann3.span)])

Within Ann1:  ['Ann10', 'Ann5', 'Ann3', 'Ann7', 'Ann9', 'Ann11', 'Ann6', 'Ann8', 'Ann12']
Coextensive with Ann3: ['Ann9']
Coextensive with span of Ann3: ['Ann3', 'Ann9']


# Document: loading/saving

Supported formats:
* plain text
* bdocjs, bdocym, bdocmp: load/save JSON, YAML, MsgPack (aliasing: only bdocym)
* GATE xml: load (but only basic data types, no aliasing)
* HTML: load and create annotations for HTML entities
* plain text: load / save
* tweet: load v1 format, WIP!
* pickle: load/save
* html-ann-viewer: save HTML stand-alone file or HTML section for notebooks

## Document: view sets/types

Use: `doc.show(annsets=["set1", ("set2", "type1"), ("set3", ["type1", "type2"])]`

In [40]:
doc2.show(annsets=[("Original markups", ["h1","h2","a","li"])])

## Exchange Documents with Java GATE

* Python GateNLP can read Java GATE XML format
* GATE plugin [Format_Bdoc](https://gatenlp.github.io/gateplugin-Format_Bdoc/) provides support for loading/saving formats bdocjs, bdocym and bdocmp in Java GATE
* Offsets differ between GATE and GateNLP: 
  * Java: offsets refer to UTF-16 encoding, possibly a *surrogate pair* of UTF-16 characters
  * Python: offsets refer to Unicode code points 
  * bdocjs/bdocym/bdocmp automatically convert the offsets on either side
  * field `offset_type` is either `p` or `j` 

# Corpus

* a list-like collection of a fixed number of documents which can be retrieved and stored by index:<br>
  get: `doc = corpus[2]` set: `corpus[3] = doc`
* on retrieval, the index gets stored in a document feature
* implements `store(doc)` to save a document to the index stored in the document feature
  * !! Important for batched pipelining !!
* some implementations: `append(doc)` to add a new document to the corpus
* some implementations: store/retrieve `None`
  * on retrieveal: `None` indicates absence of document
  * on storing: `None` indicates that document should get removed or should not get updated


## DirFilesCorpus

* all (recursive) files in a directory with some specific extension
* specify some specific format or infer from file extension
* stores the relative file path as a document feature


In [33]:
from gatenlp.corpora import DirFilesCorpus
corp1 = DirFilesCorpus("data/dir1")  # get all the matching filenames from the directory
print("Number of documents:", len(corp1))
doc1 = corp1[2]  # actually read the document from the directory
print("Text for idx=2:", doc1.text)
print("Features for idx=2:", doc1.features)
doc1.annset().add(0,len(doc1.text), "Document", dict(what="test document"))
# this writes the document back to the file:
corp1.store(doc1)
# could also have used: corp1[2] = doc1


Number of documents: 4
Text for idx=2: This is another document for testing which mentions John Smith.
Features for idx=2: Features({'gate.SourceURL': 'created from String', '__idx_140684144323216': 2, '__idx': 2, '__relpath': 'doc2.bdocjs', '__abspath': 'data/dir1/doc2.bdocjs'})


## Other Corpus Classes

* `ListCorpus`: wrap Python list-like collection
* `NumberedDirFilesCorpus`: create a directory tree where the path represents digits of a large number
  * e.g. `000/002/341.bdoc` for element number 2341 of 600000000 total
* `EveryNthCorpus`: wrap a corpus and access only elements $k*i + o$ for $i = 0..\lfloor(n/k)\rfloor$
  * $k$: every that many elements
  * $o$: start with this element ($o < k$)
  * e.g.: get elements 3, 7, 11, 15 from a corpus with 17 elements
  * useful for processing files in a DirFilesCorpus with multiple processes
* `ShuffledCorpus`: random re-ordering of the elements in the wrapped corpus
* `CachedCorpus`: store retrieved elements from a (slow) base corpus in a (fast) cache corpus
* Still work in progress

## Source, Destination

* Document Source: something that can be iterated over to get one Document after the other
  * unknown size
  * a Corpus may also function as a Source
* Document Destination: something that has `append(doc)` to add Document instances
  * unknown final size
  * also has `close()` to end writing
  * may implement the `with documentdestination as dest:` pattern
  * an appendable Corpus may also function as a Destination


## Source, Destination examples

* `BdocjsLinesFileSource/Destination`: one line of bdocjs serialization per document
* `TsvFileSource`: one column in a TSV file contains the text, other columns can be stored in features
* `PandasDfSource`: similar to TSV source, but for a Pandas data frame
* `ConllUFileSource`: corpus from a (large) ConllU File
* work in progress

## Conll-U Source

* Read in one of the many multilingual corpora from https://universaldependencies.org/
* create documents from k sentences, paragraphs conll documents
* use original text hints or space hints, if available
* Example: first few lines of `ar-ud-train.conllu`

In [36]:
from gatenlp.corpora.conll import ConllUFileSource
src = ConllUFileSource("data/ar-tiny.conllu", group_by="doc", n=1)
corp = list(src)
print(len(corp))

3


## Conll-U Source

In [37]:
corp[0].show(doc_style="direction: rtl; font-size: 1.5em; line-height: 1.5;")

# Annotators, Executors

* Annotator: a *callable* that accepts a document to process and either:
  * returns the same or a different document (most common situation)
  * returns None: something went wrong or the document should get filtered
  * returns a list of zero to n documents: filter, error, split documents
  * standard methods for handling over-a-corpus results
  * `pipe` method for batched processing
* Pipeline: a special annotator that recursively runs other annotators in sequence
* Executor: a class that runs an annotator
  * on a corpus
  * on a source and optional destination
  * takes care of handling None, lists of returned documents

 



# Spacy Annotator

* Use a [SpaCy](https://spacy.io/) pipeline to annotate a document
* convert spacy tokens, entities etc into Annotations, convert token attributes into annotation features
* makes it much easier to add own annotations and features, no need to keep vocab files around
* but possibly not as optimized/fast as Spacy

Preparation:
* make sure spacy dependency is installed for your gatenlp environment: <br> 
`pip install -U spacy` (not necessary if `gatenlp[all]` was used)
* make sure the [model for the language](https://spacy.io/models) is installed:<br>
  English: `python -m spacy download en_core_web_sm`
* To use in notebook, need to restart kernel after installation! 

## Spacy Annotator


In [41]:
import spacy
print("Spacy version:", spacy.__version__)
from gatenlp.lib_spacy import AnnSpacy

nlp = spacy.load("en_core_web_sm")
annotator = AnnSpacy(pipeline=nlp, outsetname="Spacy")
doc2.annset("Spacy").clear()   # avoid annotation duplication when running several times
doc2 = annotator(doc2)


Spacy version: 2.3.2


## Spacy Annotator

In [42]:
# Adapt size of viewer
from IPython.core.display import display, HTML
display(HTML("<style>#view2-wrapper { font-size: 80% !important; } #view2-row1 {height: 15em; min-height: 5em;}</style>"))


doc2.show(htmlid="view2")

# Stanza Annotator

* Use a [Stanza](https://stanfordnlp.github.io/stanza/) pipeline to annotate a document
* convert stanza tokens, entities etc into Annotations, convert token attributes into annotation features

Preparation:
* make sure stanza dependency is installed for your gatenlp environment: <br>
`pip install -U stanza` (not necessary if `gatenlp[all]` was used)
* make sure the [model for the language](https://spacy.io/models) is installed:<br>
  English: `python -c "import stanza; stanza.download('en')" `

## Stanza Annotator

In [43]:
import stanza
print("Stanza version:",stanza.__version__)
from gatenlp.lib_stanza import AnnStanza

nlpstanza = stanza.Pipeline(logging_level="ERROR")
annotatorstanza = AnnStanza(pipeline=nlpstanza, outsetname="Stanza")
doc2.annset("Stanza").clear()   # avoid annotation duplication when running several times
doc2 = annotatorstanza(doc2)


Stanza version: 1.2


## Stanza Annotator

In [44]:
# change size of document viewer
from IPython.core.display import display, HTML
display(HTML("<style>#view3-wrapper { font-size: 100% !important; } #view3-row1 {height: 10em; min-height: 5em;}</style>"))

doc2.show(htmlid="view3")

# Gazetteers

* Look up tokens/words/strings/phrases in a long list ("gazetteer list")
* Still work in progress
* Implemented: first version of **TokenGazetter**
  * Match sequences of tokens
  * Tokens in documents: based on sequences of annotations
  * Use underlying document text or some feature value per annotation ("Token")
  * gazetteer list: each entry should be a list of tokens as well!


# Document Tokenization

* Stanza and Spacy both produce Tokens
* Also possible to use NLTKTokenizer: wrap any of the NLTK tokenizers

In [45]:
from gatenlp.processing.tokenizer import NLTKTokenizer
from nltk.tokenize.destructive import NLTKWordTokenizer  # get some tokenizer to use
nltk_tokenizer = NLTKTokenizer(nltk_tokenizer=NLTKWordTokenizer(), token_type="Token")
doc2.annset().clear()  
doc2 = nltk_tokenizer(doc2)

## Document Tokenization

In [46]:
doc2

Document(This is a test document.

It contains just a few sentences. 
Here is a sentence that mentions a few named entities like 
the persons Barack Obama or Ursula von der Leyen, locations
like New York City, Vienna or Beijing or companies like 
Google, UniCredit or Huawei. And here is Donald Trump, it may not be the real one :P

Lets say Boris Johnson aka Bojo tweets from his BorisJohnson account, would be nice to match them!

Here we include a URL https://gatenlp.github.io/python-gatenlp/ 
and a fake email address john.doe@hiscoolserver.com as well 
as #some #cool #hastags and a bunch of emojis like 😽 (a kissing cat),
👩‍🏫 (a woman teacher), 🧬 (DNA), 
🧗 (a person climbing), 

Here we test a few different scripts, e.g. Hangul 한글 or 
simplified Hanzi 汉字 or Farsi فارسی and Arabic ,اَلْعَرَبِيَّةُ, which goes from right to left.

,features=Features({}),anns={'Spacy': AnnotationSet([Annotation(0,4,Token,features=Features({'_i': 0, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': False, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': True, 'is_space': False, 'is_stop': True, 'is_title': True, 'is_upper': False, 'lang': 'en', 'lemma': 'this', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 12943039165150086467, 'pos': 'DET', 'prefix': 'T', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'DT', 'shape': 'Xxxx', 'suffix': 'his', 'ent_type': '', 'dep': 'nsubj', 'head': 2, 'left_edge': 0, 'right_edge': 0}),id=0), Annotation(0,26,Sentence,features=Features({}),id=335), Annotation(4,5,SpaceToken,features=Features({'is_space': True}),id=1), Annotation(5,7,Token,features=Features({'_i': 1, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': True, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'be', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 3411606890003347522, 'pos': 'AUX', 'prefix': 'i', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'VBZ', 'shape': 'xx', 'suffix': 'is', 'ent_type': '', 'dep': 'ROOT', 'head': 2, 'left_edge': 0, 'right_edge': 10}),id=2), Annotation(7,8,SpaceToken,features=Features({'is_space': True}),id=3), Annotation(8,9,Token,features=Features({'_i': 2, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': True, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'a', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 11901859001352538922, 'pos': 'DET', 'prefix': 'a', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'DT', 'shape': 'x', 'suffix': 'a', 'ent_type': '', 'dep': 'det', 'head': 8, 'left_edge': 4, 'right_edge': 4}),id=4), Annotation(8,23,NounChunk,features=Features({}),id=345), Annotation(9,10,SpaceToken,features=Features({'is_space': True}),id=5), Annotation(10,14,Token,features=Features({'_i': 3, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': False, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'test', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 1618900948208871284, 'pos': 'NOUN', 'prefix': 't', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'NN', 'shape': 'xxxx', 'suffix': 'est', 'ent_type': '', 'dep': 'compound', 'head': 8, 'left_edge': 6, 'right_edge': 6}),id=6), Annotation(14,15,SpaceToken,f

# Gazetteer List

1) Use prepared list with already tokenized entries


In [47]:
from gatenlp.processing.gazetteer import TokenGazetteer
gazlist1 = [
    (["Donald", "Trump"], dict(what="person", country="US")),
    (["Boris", "Johnson"], dict(what="person", country="UK")),
    (["Google"], dict(what="company", country="Everywhere, really!"))    
]
tgaz1 = TokenGazetteer(gazlist1, fmt="gazlist", annset="", outset="TGaz1", outtype="Lookup")
doc2.annset("TGaz1").clear()
doc2 = tgaz1(doc2)

## Gazetteer List

Result when using the prepared list:

In [48]:
doc2.show()

# Gazetteer List

2) Load a list from a file, using JAVA GATE "def" format (https://gate.ac.uk/userguide/sec:annie:gazetteer):

* A somthing.def file contains one line for each list file to use
* Each somename.lst file contains one line with entry to match and arbitrary features

E.g. `data/gaz1.def`:
```
persons.lst
companies.lst
```

`data/persons.lst`:
```
Donald Trump    what=person     country=US
Boris Johnson   what=person     country=UK
```

`data/companies.lst`:
```
Google  where=Everywhere, really!
```


## Gazetteer List (GATE def)

In [49]:
tgaz2 = TokenGazetteer("data/gaz1.def", fmt="gate-def", annset="", outset="TGaz2", outtype="Lookup", tokenizer=nltk_tokenizer)
doc2.annset("TGaz2").clear()
doc2 = tgaz2(doc2)

2021-02-26 11:24:17,656|INFO|gatenlp.processing.gazetteer|Reading list file data/persons.lst
2021-02-26 11:24:17,657|INFO|gatenlp.processing.gazetteer|Reading list file data/companies.lst


## Gazetteer List

Result when using the loaded GATE-style gazetteer files:

In [50]:
doc2

Document(This is a test document.

It contains just a few sentences. 
Here is a sentence that mentions a few named entities like 
the persons Barack Obama or Ursula von der Leyen, locations
like New York City, Vienna or Beijing or companies like 
Google, UniCredit or Huawei. And here is Donald Trump, it may not be the real one :P

Lets say Boris Johnson aka Bojo tweets from his BorisJohnson account, would be nice to match them!

Here we include a URL https://gatenlp.github.io/python-gatenlp/ 
and a fake email address john.doe@hiscoolserver.com as well 
as #some #cool #hastags and a bunch of emojis like 😽 (a kissing cat),
👩‍🏫 (a woman teacher), 🧬 (DNA), 
🧗 (a person climbing), 

Here we test a few different scripts, e.g. Hangul 한글 or 
simplified Hanzi 汉字 or Farsi فارسی and Arabic ,اَلْعَرَبِيَّةُ, which goes from right to left.

,features=Features({}),anns={'Spacy': AnnotationSet([Annotation(0,4,Token,features=Features({'_i': 0, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': False, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': True, 'is_space': False, 'is_stop': True, 'is_title': True, 'is_upper': False, 'lang': 'en', 'lemma': 'this', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 12943039165150086467, 'pos': 'DET', 'prefix': 'T', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'DT', 'shape': 'Xxxx', 'suffix': 'his', 'ent_type': '', 'dep': 'nsubj', 'head': 2, 'left_edge': 0, 'right_edge': 0}),id=0), Annotation(0,26,Sentence,features=Features({}),id=335), Annotation(4,5,SpaceToken,features=Features({'is_space': True}),id=1), Annotation(5,7,Token,features=Features({'_i': 1, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': True, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'be', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 3411606890003347522, 'pos': 'AUX', 'prefix': 'i', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'VBZ', 'shape': 'xx', 'suffix': 'is', 'ent_type': '', 'dep': 'ROOT', 'head': 2, 'left_edge': 0, 'right_edge': 10}),id=2), Annotation(7,8,SpaceToken,features=Features({'is_space': True}),id=3), Annotation(8,9,Token,features=Features({'_i': 2, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': True, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'a', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 11901859001352538922, 'pos': 'DET', 'prefix': 'a', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'DT', 'shape': 'x', 'suffix': 'a', 'ent_type': '', 'dep': 'det', 'head': 8, 'left_edge': 4, 'right_edge': 4}),id=4), Annotation(8,23,NounChunk,features=Features({}),id=345), Annotation(9,10,SpaceToken,features=Features({'is_space': True}),id=5), Annotation(10,14,Token,features=Features({'_i': 3, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': False, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'test', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 1618900948208871284, 'pos': 'NOUN', 'prefix': 't', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'NN', 'shape': 'xxxx', 'suffix': 'est', 'ent_type': '', 'dep': 'compound', 'head': 8, 'left_edge': 6, 'right_edge': 6}),id=6), Annotation(14,15,SpaceToken,f

# Token Gazetteer

Other features:

* Match each gazetteer token against the feature of a document token, not the underlying document text (e.g. the lemma)
* only match within another annotation type (e.g. only in noun phrases)
* More to come: still WIP!
  * handle space tokens
  * handle split tokens (do not cross them!)

Planned: String Gazetteer

* Match sequences of characters 
* match within words (e.g. composita)


# PAMPAC

* PAttern Matching through PArser Combinators
* A pattern language for matching annotations and text
* DSL (domain specific language) implemented as Python classes
* Purpose: similar to Java GATE JAPE / JAPE Plus, but:
  * does not need a separate language
  * more powerful (e.g. directly match text AND annotations)



## PAMPAC - How to use

* define **Rules**
* each Rule consists of:
  * a **Pattern** to match in the document
  * an **Action** to perform if the pattern matches
* create a **Pampac** instance from the rules
* **run** the Pampac instance on the document, using a set of annotations and specify the output annotation set
  

## PAMPAC - Example 1

Lets create a rule that annotates any Token which is within a PERSON or ORG annotation:

In [51]:
from gatenlp.pam.pampac import Ann, AnnAt, Rule, Pampac, AddAnn, N, Seq, Or
from gatenlp.pam.matcher import FeatureMatcher, ifnot

r1 = Rule(
    # first the pattern
    Or ( Ann("Token", name="tok").within("ORG"),
         Ann("Token", name="tok").within("PERSON")
       ),
    # then the action for the pattern
    AddAnn(name="tok", anntype="PersOrOrg")
)
# get the annotations we want to use for matching
anns2match = doc2.annset("Stanza").with_type(["Token", "PERSON", "ORG"])

outset = doc2.annset("Pampac1")
outset.clear()
# Create the Pampac instance from the single rule and run it on the annotations, also specify output set
# The run method returns the list of offsets and the action return values where the rule matches in the doc
Pampac(r1).run(doc2, anns2match, outset=outset)
len(outset)

15

## PAMPAC - Example 1



In [52]:
doc2

Document(This is a test document.

It contains just a few sentences. 
Here is a sentence that mentions a few named entities like 
the persons Barack Obama or Ursula von der Leyen, locations
like New York City, Vienna or Beijing or companies like 
Google, UniCredit or Huawei. And here is Donald Trump, it may not be the real one :P

Lets say Boris Johnson aka Bojo tweets from his BorisJohnson account, would be nice to match them!

Here we include a URL https://gatenlp.github.io/python-gatenlp/ 
and a fake email address john.doe@hiscoolserver.com as well 
as #some #cool #hastags and a bunch of emojis like 😽 (a kissing cat),
👩‍🏫 (a woman teacher), 🧬 (DNA), 
🧗 (a person climbing), 

Here we test a few different scripts, e.g. Hangul 한글 or 
simplified Hanzi 汉字 or Farsi فارسی and Arabic ,اَلْعَرَبِيَّةُ, which goes from right to left.

,features=Features({}),anns={'Spacy': AnnotationSet([Annotation(0,4,Token,features=Features({'_i': 0, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': False, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': True, 'is_space': False, 'is_stop': True, 'is_title': True, 'is_upper': False, 'lang': 'en', 'lemma': 'this', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 12943039165150086467, 'pos': 'DET', 'prefix': 'T', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'DT', 'shape': 'Xxxx', 'suffix': 'his', 'ent_type': '', 'dep': 'nsubj', 'head': 2, 'left_edge': 0, 'right_edge': 0}),id=0), Annotation(0,26,Sentence,features=Features({}),id=335), Annotation(4,5,SpaceToken,features=Features({'is_space': True}),id=1), Annotation(5,7,Token,features=Features({'_i': 1, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': True, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'be', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 3411606890003347522, 'pos': 'AUX', 'prefix': 'i', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'VBZ', 'shape': 'xx', 'suffix': 'is', 'ent_type': '', 'dep': 'ROOT', 'head': 2, 'left_edge': 0, 'right_edge': 10}),id=2), Annotation(7,8,SpaceToken,features=Features({'is_space': True}),id=3), Annotation(8,9,Token,features=Features({'_i': 2, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': True, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'a', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 11901859001352538922, 'pos': 'DET', 'prefix': 'a', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'DT', 'shape': 'x', 'suffix': 'a', 'ent_type': '', 'dep': 'det', 'head': 8, 'left_edge': 4, 'right_edge': 4}),id=4), Annotation(8,23,NounChunk,features=Features({}),id=345), Annotation(9,10,SpaceToken,features=Features({'is_space': True}),id=5), Annotation(10,14,Token,features=Features({'_i': 3, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': False, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'test', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 1618900948208871284, 'pos': 'NOUN', 'prefix': 't', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'NN', 'shape': 'xxxx', 'suffix': 'est', 'ent_type': '', 'dep': 'compound', 'head': 8, 'left_edge': 6, 'right_edge': 6}),id=6), Annotation(14,15,SpaceToken,f

## PAMPAC - Example 2

Create a rule that annotates any Sequence of two or more Token annotations which have a "upos" tag of "PROPN", separated by at most one other arbitrary token:

In [53]:
from gatenlp.pam.pampac import Ann, AnnAt, Rule, Pampac, AddAnn, N, Seq
from gatenlp.pam.matcher import FeatureMatcher, ifnot

feat = FeatureMatcher(upos="PROPN")
r1 = Rule(
    # first the pattern
    Seq( Ann("Token", features=feat),
         N( Seq( N(Ann("Token", features=ifnot(feat)), min=0, max=1),
                 Ann("Token", features=feat)),
           min=1, max=99),
         name="seq1"
       ),
    # then the action for the pattern
    AddAnn(name="seq1", anntype="PROPNSEQ")
)
# get the annotations we want to use for matching
anns2match = doc2.annset("Stanza").with_type("Token")

outset = doc2.annset("Pampac2")
outset.clear()
# Create the Pampac instance from the single rule and run it on the annotations, also specify output set
# The run method returns the list of offsets and the action return values where the rule matches in the doc
Pampac(r1).run(doc2, anns2match, outset=outset)
len(outset)

8

## PAMPAC - Example 2

Result: found 8 matches and added annotations for them:

In [54]:
doc2

Document(This is a test document.

It contains just a few sentences. 
Here is a sentence that mentions a few named entities like 
the persons Barack Obama or Ursula von der Leyen, locations
like New York City, Vienna or Beijing or companies like 
Google, UniCredit or Huawei. And here is Donald Trump, it may not be the real one :P

Lets say Boris Johnson aka Bojo tweets from his BorisJohnson account, would be nice to match them!

Here we include a URL https://gatenlp.github.io/python-gatenlp/ 
and a fake email address john.doe@hiscoolserver.com as well 
as #some #cool #hastags and a bunch of emojis like 😽 (a kissing cat),
👩‍🏫 (a woman teacher), 🧬 (DNA), 
🧗 (a person climbing), 

Here we test a few different scripts, e.g. Hangul 한글 or 
simplified Hanzi 汉字 or Farsi فارسی and Arabic ,اَلْعَرَبِيَّةُ, which goes from right to left.

,features=Features({}),anns={'Spacy': AnnotationSet([Annotation(0,4,Token,features=Features({'_i': 0, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': False, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': True, 'is_space': False, 'is_stop': True, 'is_title': True, 'is_upper': False, 'lang': 'en', 'lemma': 'this', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 12943039165150086467, 'pos': 'DET', 'prefix': 'T', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'DT', 'shape': 'Xxxx', 'suffix': 'his', 'ent_type': '', 'dep': 'nsubj', 'head': 2, 'left_edge': 0, 'right_edge': 0}),id=0), Annotation(0,26,Sentence,features=Features({}),id=335), Annotation(4,5,SpaceToken,features=Features({'is_space': True}),id=1), Annotation(5,7,Token,features=Features({'_i': 1, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': True, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'be', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 3411606890003347522, 'pos': 'AUX', 'prefix': 'i', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'VBZ', 'shape': 'xx', 'suffix': 'is', 'ent_type': '', 'dep': 'ROOT', 'head': 2, 'left_edge': 0, 'right_edge': 10}),id=2), Annotation(7,8,SpaceToken,features=Features({'is_space': True}),id=3), Annotation(8,9,Token,features=Features({'_i': 2, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': True, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'a', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 11901859001352538922, 'pos': 'DET', 'prefix': 'a', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'DT', 'shape': 'x', 'suffix': 'a', 'ent_type': '', 'dep': 'det', 'head': 8, 'left_edge': 4, 'right_edge': 4}),id=4), Annotation(8,23,NounChunk,features=Features({}),id=345), Annotation(9,10,SpaceToken,features=Features({'is_space': True}),id=5), Annotation(10,14,Token,features=Features({'_i': 3, 'is_alpha': True, 'is_bracket': False, 'is_currency': False, 'is_digit': False, 'is_left_punct': False, 'is_lower': True, 'is_oov': True, 'is_punct': False, 'is_quote': False, 'is_right_punct': False, 'is_sent_start': None, 'is_space': False, 'is_stop': False, 'is_title': False, 'is_upper': False, 'lang': 'en', 'lemma': 'test', 'like_email': False, 'like_num': False, 'like_url': False, 'orth': 1618900948208871284, 'pos': 'NOUN', 'prefix': 't', 'prob': -20.0, 'rank': 18446744073709551615, 'sentiment': 0.0, 'tag': 'NN', 'shape': 'xxxx', 'suffix': 'est', 'ent_type': '', 'dep': 'compound', 'head': 8, 'left_edge': 6, 'right_edge': 6}),id=6), Annotation(14,15,SpaceToken,f

# GATE Worker

* Allows running the Java GATE process from Python
* API for exchanging document and performing frequent Java GATE tasks from Python
* [Py4J](https://www.py4j.org/) API to run ANY Java from Python
* Python connects to a Java process, communicates over sockets
  * Option 1: Start GATE GUI, load PythonWorkerLr, then connect a GateNLP GateWorker to it
  * Option 2: Start Java GATE worker using the `gatenlp-gate-worker` command
  * Option 3: directly start the Java GATE worker when creating the GateNLP GateWorker instance

Let's try Option 3 first: `GATE_HOME` environment variable must be set, or must know GATE installation directory

## GATE Worker

In [55]:
from gatenlp.gateworker import GateWorker

gs = GateWorker()
# if GATE_HOME not set use gs = GateWorker(gatehome="/where/Gate/is/Installed")
# if java is not on the PATH use gs = GateWorker(java=""/path/to/the/java/binary")

Trying to start GATE Worker on port=25333 host=127.0.0.1 log=false keep=false
PythonWorkerRunner.java: starting server with 25333/127.0.0.1/Sj9KIi9elOfJp9eupily-ELd3mM/false


In [56]:
# Create a GATE document on the JAVA GATE side and return a handle
gdoc1 = gs.createDocument("An example document mentioning Barack Obama and New York")
# Can call Java API methods on that handle and get/convert the result
print(gdoc1.getClass())
print(gdoc1.getName())
print(gdoc1.getAnnotationSetNames())

class gate.corpora.DocumentImpl
GATE Document_00015
set()


## GATE Worker

In [57]:
# lets load the prepared ANNIE pipeline on the Java side and process the GATE document with it
gs.loadMavenPlugin("uk.ac.gate.plugins", "annie", "9.0")
gpipe = gs.loadPipelineFromPlugin("uk.ac.gate.plugins", "annie", "/resources/ANNIE_with_defaults.gapp")
gcorp = gs.newCorpus()
gcorp.add(gdoc1)
gpipe.setCorpus(gcorp)
gpipe.execute()

## GATE Worker

So far, everything happened on the Java side, use a GateWorker API method to convert the document into a Python GateNLP document:

In [58]:
pdoc1 = gs.gdoc2pdoc(gdoc1)
pdoc1

Document(An example document mentioning Barack Obama and New York,features=Features({'gate.SourceURL': 'created from String'}),anns={'': AnnotationSet([Annotation(0,2,Token,features=Features({'orth': 'upperInitial', 'string': 'An', 'kind': 'word', 'length': '2', 'category': 'DT'}),id=0), Annotation(0,56,Sentence,features=Features({}),id=21), Annotation(2,3,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=1), Annotation(3,10,Token,features=Features({'orth': 'lowercase', 'string': 'example', 'kind': 'word', 'length': '7', 'category': 'NN'}),id=2), Annotation(10,11,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=3), Annotation(11,19,Token,features=Features({'orth': 'lowercase', 'string': 'document', 'kind': 'word', 'length': '8', 'category': 'NN'}),id=4), Annotation(19,20,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=5), Annotation(20,30,Token,features=Features({'orth': 'lowercase', 'string': 'mentioning', 'kind': 'word', 'length': '10', 'category': 'VBG'}),id=6), Annotation(30,31,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=7), Annotation(31,37,Token,features=Features({'orth': 'upperInitial', 'string': 'Barack', 'kind': 'word', 'length': '6', 'category': 'NNP'}),id=8), Annotation(31,43,Lookup,features=Features({'majorType': 'person_full', 'gender': 'male'}),id=17), Annotation(31,43,Person,features=Features({'firstName': 'Barack', 'ruleFinal': 'PersonFinal', 'gender': 'male', 'surname': 'Obama', 'kind': 'fullName', 'rule': 'GazPerson'}),id=29), Annotation(37,38,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=9), Annotation(38,43,Token,features=Features({'orth': 'upperInitial', 'string': 'Obama', 'kind': 'word', 'length': '5', 'category': 'NNP'}),id=10), Annotation(38,43,Lookup,features=Features({'majorType': 'person_full', 'gender': 'male'}),id=18), Annotation(43,44,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=11), Annotation(44,47,Token,features=Features({'orth': 'lowercase', 'string': 'and', 'kind': 'word', 'length': '3', 'category': 'CC'}),id=12), Annotation(47,48,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=13), Annotation(48,51,Token,features=Features({'orth': 'upperInitial', 'string': 'New', 'kind': 'word', 'length': '3', 'category': 'NNP'}),id=14), Annotation(48,56,Lookup,features=Features({'majorType': 'location', 'minorType': 'city'}),id=19), Annotation(48,56,Location,features=Features({'ruleFinal': 'LocFinal', 'rule': 'Location1', 'locType': 'city'}),id=30), Annotation(51,52,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=15), Annotation(52,56,Token,features=Features({'orth': 'upperInitial', 'string': 'York', 'kind': 'word', 'length': '4', 'category': 'NNP'}),id=16), Annotation(52,56,Lookup,features=Features({'majorType': 'location', 'minorType': 'city'}),id=20)])})

## GateWorker

* Stopping: the GateWorker (Java process) can get stopped using `gs.close()`
* Will also automatically stop when the Python process ends

In [59]:
gs.close()

# GateWorker Annotator

An annotator to process Python GateNLP documents with a Java GATE pipeline

In [60]:
from gatenlp.gateworker import GateWorkerAnnotator
# Specify a prepared GATE pipeline file to get loaded into Java GATE
# optionally add the gatehome=... kw argument
# optionally specify port using port=23445 or similar
gs_app = GateWorkerAnnotator(pipeline="data/annie.xgapp", port=25444)

Trying to start GATE Worker on port=25444 host=127.0.0.1 log=false keep=false
PythonWorkerRunner.java: starting server with 25444/127.0.0.1/St7EdWl-EcnksuhuGfSingmtulg/false


## GateWorkerAnnotator

Example, running on a directory corpus:

In [61]:
dircorpus = DirFilesCorpus("data/dir1", sort=True)
exe = SerialCorpusExecutor(annotator=gs_app, corpus=dircorpus)
exe()
gs_app.close()

tmpdoc = dircorpus[2]
print(tmpdoc.features)
tmpdoc

Features({'gate.SourceURL': 'created from String', '__idx_140684144569144': 2, '__idx': 2, '__relpath': 'doc3.bdocjs', '__abspath': 'data/dir1/doc3.bdocjs'})


Document(A third document for testing,features=Features({'gate.SourceURL': 'created from String', '__idx_140684144569144': 2, '__idx': 2, '__relpath': 'doc3.bdocjs', '__abspath': 'data/dir1/doc3.bdocjs'}),anns={'': AnnotationSet([Annotation(0,1,Token,features=Features({'orth': 'upperInitial', 'string': 'A', 'kind': 'word', 'length': '1', 'category': 'DT'}),id=253), Annotation(0,28,Sentence,features=Features({}),id=263), Annotation(1,2,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=254), Annotation(2,7,Token,features=Features({'orth': 'lowercase', 'string': 'third', 'kind': 'word', 'length': '5', 'category': 'JJ'}),id=255), Annotation(2,7,Lookup,features=Features({'majorType': 'date', 'minorType': 'ordinal'}),id=262), Annotation(7,8,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=256), Annotation(8,16,Token,features=Features({'orth': 'lowercase', 'string': 'document', 'kind': 'word', 'length': '8', 'category': 'NN'}),id=257), Annotation(16,17,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=258), Annotation(17,20,Token,features=Features({'orth': 'lowercase', 'string': 'for', 'kind': 'word', 'length': '3', 'category': 'IN'}),id=259), Annotation(20,21,SpaceToken,features=Features({'string': ' ', 'kind': 'space', 'length': '1'}),id=260), Annotation(21,28,Token,features=Features({'orth': 'lowercase', 'string': 'testing', 'kind': 'word', 'length': '7', 'category': 'NN'}),id=261)])})

In [66]:
# NOT SHOWN: clean the corpus we have just processed
def clearanns(doc):
    # instead of just removing annotations, return a completely new document
    docnew = Document(doc.text)
    return docnew
annclear = AnnotatorFunction(clearanns)
execlear = SerialCorpusExecutor(annotator=annclear, corpus=dircorpus)
execlear()
dircorpus[1]

Document(This is another document for testing which mentions John Smith.,features=Features({'__idx_140684144569144': 1, '__idx': 1, '__relpath': 'doc2.bdocjs', '__abspath': 'data/dir1/doc2.bdocjs'}),anns={})